In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//div[@class="productView-img-container"]/a/@href')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_dt = html.xpath('//dl[@class="productView-info"]/dt/text()')
                list_dd = html.xpath('//dl[@class="productView-info"]/dd/text()')
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="tabDescription-content"]/table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Title': title,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])

                for dt, dd in zip(list_dt, list_dd):
                    df_temp.loc[0, dt.strip()[:-1].lower()] = dd.strip()

                for tr in list_tr:
                    if tr.xpath('./td[1]/text()')[0].strip() == 'Application':
                        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = '\n'.join(tr.xpath('./td[2]/p/text()')).strip()
                    else:
                        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = ' '.join(tr.xpath('./td[2]/text()')).strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：355

https://rotomaster.com/turbochargers/reman-turbo-for-2000-2005-27l-manual-trans-audi-k8030117r/  <->  [ok] - 剩余数量：335 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2000-2001-18l-audi-volkswagen-k8030135r/  <->  [ok] - 剩余数量：334 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2000-2005-27l-manual-trans-audi-k8030116r/  <->  [ok] - 剩余数量：333 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2000-2002-59l-cummins-automatic-trans-dodge-h8350109r/  <->  [ok] - 剩余数量：332 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2001-2001-18l-audi-k8030141r/  <->  [ok] - 剩余数量：331 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2000-2006-18l-audi-volkswagen-k8030129r/  <->  [ok] - 剩余数量：330 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2002-2002-18l-audi-k8030122r/  <->  [ok] - 剩余数量：329 - 当前时间：09:57:29
https://rotomaster.com/turbochargers/reman-turbo-for-2002-2002-27l-righ

https://rotomaster.com/turbochargers/reman-turbo-for-1996-2002-65l-chevrolet-gmc-j8650106r/  <->  [ok] - 剩余数量：274 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-1991-1999-30l-right-side-dodge-mitsubishi-m8040109r/  <->  [ok] - 剩余数量：273 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-1994-1995-59l-cummins-dodge-h8350103r/  <->  [ok] - 剩余数量：272 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-1996-1998-59l-cummins-dodge-h8350131r/  <->  [ok] - 剩余数量：271 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/replacement-turbo2008-2021-2-5l-subaru-impreza-wrx-sti-j1520102n/  <->  [ok] - 剩余数量：270 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-1995-1998-59l-cummins-automatic-trans-dodge-h8350104r/  <->  [ok] - 剩余数量：269 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-1996-2004-65l-am-hummer-j8650107r/  <->  [ok] - 剩余数量：268 - 当前时间：09:57:36
https://rotomaster.com/turbochargers/reman-turbo-for-

https://rotomaster.com/turbochargers/reman-turbo-for-2008-2017-20l-audi-volkswagen-j8050114r/  <->  [ok] - 剩余数量：212 - 当前时间：09:57:46
https://rotomaster.com/turbochargers/reman-turbo-for-2010-2012-20l-hyundai-m8040123r/  <->  [ok] - 剩余数量：211 - 当前时间：09:57:46
https://rotomaster.com/products/turbochargers/reman-turbo-for-2011-2015-BMW-3-0l-s8030180r/  <->  [ok] - 剩余数量：210 - 当前时间：09:57:46
https://rotomaster.com/products/turbochargers/reman-turbo-for-2014-2022-hyundai-2-0l-m8030181r/  <->  [ok] - 剩余数量：209 - 当前时间：09:57:46
https://rotomaster.com/turbochargers/reman-turbo-for-2013-2018-20l-bmw-m8040177r/  <->  [ok] - 剩余数量：208 - 当前时间：09:57:46
https://rotomaster.com/turbochargers/reman-turbo-for-2011-2011-16l-nissan-m8030108r/  <->  [ok] - 剩余数量：207 - 当前时间：09:57:46
https://rotomaster.com/turbochargers/reman-turbo-for-2013-2019-16l-ford-s8390160r/  <->  [ok] - 剩余数量：206 - 当前时间：09:57:47
https://rotomaster.com/turbochargers/reman-turbo-for-2013-2018-35l-right-side-ecoboost-ford-lincoln-k8030164r/  <-> 

https://rotomaster.com/products/turbochargers/replacement-turbo-for-2015-2022-lexus-2-0l-j1050107n/  <->  [ok] - 剩余数量：151 - 当前时间：09:57:56
https://rotomaster.com/turbochargers/reman-turbo-for-2004-2006-27l-dodge-freightliner-mercedes-benz-a8220101r/  <->  [ok] - 剩余数量：150 - 当前时间：09:57:56
https://rotomaster.com/turbochargers/reman-turbo-for-2006-2007-66l-duramax-lbz-wturbo-vane-sensor-chevrolet-gmc-a8370105rvs/  <->  [ok] - 剩余数量：149 - 当前时间：09:57:56
https://rotomaster.com/turbochargers/reman-turbo-for-2006-2007-66l-duramax-lbz-wo-turbo-vane-sensor-chevrolet-gmc-a8370105r/  <->  [ok] - 剩余数量：148 - 当前时间：09:57:57
https://rotomaster.com/turbochargers/reman-turbo-for-2003-2011-20l-saab-m8040185r/  <->  [ok] - 剩余数量：147 - 当前时间：09:57:57
https://rotomaster.com/products/turbochargers/reman-turbo-for-2008-2018-6-4l-international-maxx-force-s8630102r/  <->  [ok] - 剩余数量：146 - 当前时间：09:57:57
https://rotomaster.com/turbochargers/reman-turbo-for-2004-2007-25l-subaru-j8550101r/  <->  [ok] - 剩余数量：145 - 当前时间：0

https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-ford-stage-2-w-66mm-88-mm-compressor-wheel-7-3l-a1380105n/  <->  [ok] - 剩余数量：89 - 当前时间：09:58:06
https://rotomaster.com/turbochargers/reman-turbo-for-1987-1988-23l-volvo-m8050106r/  <->  [ok] - 剩余数量：88 - 当前时间：09:58:06
https://rotomaster.com/turbochargers/replacement-turbo2000-2005-2-7l-right-audi-k1030157n/  <->  [ok] - 剩余数量：87 - 当前时间：09:58:06
https://rotomaster.com/turbochargers/reman-turbo-for-1982-1984-16l-renault-a8320101r/  <->  [ok] - 剩余数量：86 - 当前时间：09:58:07
https://rotomaster.com/turbochargers/replacement-turbo-for-2000-2008-detroit-diesel-series-60-127l-tmf5502-a1550115n/  <->  [ok] - 剩余数量：85 - 当前时间：09:58:07
https://rotomaster.com/turbochargers/reman-turbo-for-1984-1987-30l-nissan-a8300103r/  <->  [ok] - 剩余数量：84 - 当前时间：09:58:07
https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-7-3l-w-billet-wheel-ford-a1380152n/  <->  [ok] - 剩余数量：83 - 当前时间：09:58:08
http

https://rotomaster.com/turbochargers/replacement-turbo-for-john-deere-410c-2450-2555-26-a1340102n/  <->  [ok] - 剩余数量：28 - 当前时间：09:58:16
https://rotomaster.com/turbochargers/replacement-turbo2005-2006-1-9l-volkswagen-k1390131n/  <->  [ok] - 剩余数量：27 - 当前时间：09:58:17
https://rotomaster.com/turbochargers/replacement-turbo2003-2005-2-7l-a-t-right-audi-k1030170n/  <->  [ok] - 剩余数量：26 - 当前时间：09:58:17
https://rotomaster.com/turbochargers/replacement-turbo-for-komatsu-excavator-ktr130-9g2-q1130102n/  <->  [ok] - 剩余数量：25 - 当前时间：09:58:17
https://rotomaster.com/turbochargers/replacement-turbo2005-2009-2-0l-audi-a4-a4-quattro-k1030163n/  <->  [ok] - 剩余数量：24 - 当前时间：09:58:17
https://rotomaster.com/products/turbochargers/replacement-turbo-for-2005-2013-advance-mixer-10-8l-h1430109n/  <->  [ok] - 剩余数量：23 - 当前时间：09:58:17
https://rotomaster.com/turbochargers/replacement-turbo-for-1977-1985-180l-tl8106-a1810115n/  <->  [ok] - 剩余数量：22 - 当前时间：09:58:18
https://rotomaster.com/turbochargers/replacement-turbo-fo